# Are We Rich Yet? - Ensemble version

## This notebook attempts to split the data into training, validation, and test in order to facilitate ensemble regression later. Training is 2009-2013, Validation is 2014, and Test is 2015.

### The predictions and ROIs are stored for each classifier run on training. A final ensemble regression is run over validation results and then applied to test results.

#### I have quite the headache.

In [1]:
%matplotlib inline

%run talibref.py

### Get data

In [2]:
#df=pd.read_csv("data/IYZ.csv")
ticker = 'IYZ'
startdate=datetime.date(2006, 1, 1)
enddate=datetime.date.today()

sdatev=datetime.date(2013, 1, 1)
edatev=datetime.date(2013, 12, 31)
sdatet=datetime.date(2014, 1, 1)
edatet=datetime.date.today()

df = generate_ticker_data(ticker, startdate, enddate)
df.head()

Available data: Ticker(IYZ) from 2000.05.26 to 2015.11.27
Usable data: Ticker(IYZ) from 2001.03.14 to 2015.11.27 
Returned data: Ticker(IYZ) from 2006.01.03 to 2015.11.27 
Save path: data/IYZ_from_2006.01.03_2015.11.27.csv


,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2006-01-03,22.780001,23.020000,22.750000,22.969999,470400,23.190001,True,0.009578,23.110001,True,0.006095,1,24.338211,23.5425,22.746789,0.140258,6.759783,False,False,False,1,23.442071,23.404190,23.348186,0,1,23.223212,23.399067,23.269871,0,0,23.223027,0,27.824719,11.005720,28.393282,True,False,0,-92,0,-87.938768,1,-0.182752,-0.074972,-0.107780,0,-1.493059,0,25.489729,1,-3.446835,0,32.193851,1,36.656329,0,-82.812592,1,-75435.821127,0,22.494925,13.355293,1,100,33.333333,0,0.029218,1,22.913333,23.076666,21.606667,24.383332,21.443334,24.546665,20.136668,1
1,2006-01-04,23.000000,23.270000,22.990000,23.190001,231000,23.270000,True,0.003450,23.230000,True,0.001725,1,24.282759,23.5020,22.721241,0.300195,6.644193,False,False,False,1,23.432186,23.401353,23.346612,0,1,23.222496,23.398167,23.269442,0,0,22.750000,1,27.229864,17.429322,25.871592,True,False,0,-92,0,-38.730098,1,-0.172811,-0.094540,-0.078271,0,-1.593064,0,22.222116,1,-2.151899,0,42.210294,1,43.764472,0,-65.624949,1,5960.838222,1,46.659814,25.978497,1,100,66.666667,0,0.025809,1,23.150000,23.550001,22.080002,24.619999,21.680001,25.020000,20.610003,1
2,2006-01-05,23.219999,23.360001,23.170000,23.270000,125000,23.480000,True,0.009024,23.469999,True,0.008595,1,24.221227,23.4665,22.711774,0.369820,6.432373,False,False,False,1,23.425825,23.399613,23.345850,0,1,23.223199,23.397628,23.269445,0,0,22.750000,1,26.147041,19.139319,24.393928,True,False,0,-92,0,-8.687751,1,-0.156671,-0.106966,-0.049705,0,-1.635559,0,23.094586,1,-1.356507,0,45.370492,1,44.703667,0,-56.666703,1,40810.658297,1,71.287046,46.813928,1,100,100.000000,0,0.022553,1,23.266667,23.783334,22.313335,24.736666,21.796668,25.253333,20.843336,1
3,2006-01-06,23.219999,23.590000,23.219999,23.480000,346200,23.490000,True,0.000426,23.629999,True,0.006388,0,24.183655,23.4485,22.713346,0.521424,6.270374,False,False,False,1,23.427950,23.400678,23.347185,1,1,23.227585,23.398035,23.270856,1,0,22.774400,1,24.586401,23.741316,21.784572,True,True,1,-84,0,38.853634,1,-0.125488,-0.110670,-0.014818,0,-1.615431,0,31.182632,1,-0.381841,0,52.684956,1,48.981709,0,-39.166717,1,96014.768204,1,85.588700,67.845187,0,100,100.000000,0,0.019695,1,23.430000,24.109999,22.640000,24.899999,21.960001,25.579998,21.170001,1
4,2006-01-09,23.500000,23.500000,23.299999,23.490000,115000,23.440001,False,-0.002129,23.870001,True,0.016177,0,24.155153,23.4345,22.713848,0.538507,6.150355,False,False,False,1,23.430383,23.401861,23.348606,1,1,23.233430,23.398558,23.272517,1,0,22.823336,1,23.137236,22.349646,20.507602,True,True,1,-84,0,52.525729,1,-0.098829,-0.108302,0.009473,1,-1.589764,0,32.128396,1,1.381096,0,53.007618,1,56.538415,0,-38.333385,1,142801.488898,1,86.748587,81.208111,0,100,100.000000,0,0.017203,1,23.430000,24.109999,22.640000,24.899999,21.960001,25.579998,21.170001,1


In [3]:
df.tail()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
2489,2015-11-20,29.860001,29.900000,29.639999,29.680000,153400,29.629999,False,-0.001685,62.43750,True,1.103689,0,30.489814,29.7270,28.964186,0.469193,5.132129,False,False,False,0,29.162878,29.227878,29.312874,1,0,29.013024,29.253547,29.432540,1,0,28.890592,1,16.968452,25.622503,20.355101,False,True,1,52,1,17.310571,1,0.163694,0.224168,-0.060474,0,-0.167344,0,58.479730,0,-1.066667,0,54.495128,1,54.499672,0,-51.428571,1,385314.459605,1,88.577258,80.148257,0,72.755112,90.918371,0,0.062974,1,29.740000,31.510000,28.899999,32.350001,27.129999,34.120001,26.289998,0
2490,2015-11-23,29.690001,29.840000,29.580000,29.629999,288200,29.760000,True,0.004387,62.46875,True,1.108294,0,30.481659,29.7185,28.955341,0.442017,5.135921,False,False,False,0,29.181196,29.233204,29.316030,1,0,29.022418,29.256745,29.433563,1,0,28.951157,1,16.384234,24.429251,20.481839,False,True,1,48,1,17.048184,1,0.155769,0.210488,-0.054719,0,-0.385339,0,43.285668,0,-0.470275,0,53.636418,1,53.670164,0,-54.285771,1,285126.408665,1,77.700214,82.962588,0,49.788337,74.181150,0,0.064837,1,29.683333,31.396667,28.786666,32.293334,27.073332,34.006668,26.176665,0
2491,2015-11-24,29.549999,29.809999,29.420000,29.760000,558700,29.670000,False,-0.003024,62.00000,True,1.083333,0,30.485201,29.7370,28.988799,0.515370,5.032123,False,False,False,0,29.203895,29.240182,29.320447,1,0,29.031925,29.261214,29.435339,1,0,29.008087,1,15.342076,22.720045,21.919205,False,True,1,44,1,26.554818,1,0.158156,0.200022,-0.041866,0,-0.575353,0,41.692550,0,0.642543,0,55.595568,1,62.117478,0,-32.116838,1,350407.196714,1,73.218502,79.831991,0,77.478963,66.674137,0,0.066549,1,29.663333,31.356667,28.746666,32.273334,27.053332,33.966668,26.136665,0
2492,2015-11-25,29.770000,29.770000,29.590000,29.670000,351900,29.780001,True,0.003707,63.06250,True,1.125463,0,30.449752,29.7170,28.984248,0.467929,4.931532,False,False,False,0,29.222173,29.245875,29.323925,1,0,29.038720,29.264788,29.436697,1,0,29.061602,1,14.374358,21.956489,21.182563,False,True,1,-32,0,39.583001,1,0.151044,0.190226,-0.039182,0,-0.652351,0,48.895610,0,0.884053,0,53.897558,1,56.955565,0,-28.813559,1,332816.864851,1,64.099862,71.672859,0,10.327356,45.864885,1,0.068015,1,29.676667,31.263333,28.773333,32.166667,27.186667,33.753333,26.283333,0
2493,2015-11-27,29.650000,29.879999,29.650000,29.780001,99100,56.000000,True,0.880457,63.18750,True,1.121810,0,30.451038,29.7180,28.984962,0.542290,4.933295,False,False,False,0,29.244049,29.252949,29.328464,1,0,29.046677,29.268721,29.438715,1,0,29.111906,1,13.817867,23.099952,20.246264,False,True,1,-32,0,72.673244,1,0.152526,0.182686,-0.030161,0,-0.698883,0,51.173862,0,2.126204,0,55.679303,1,56.826208,0,-18.803333,0,299478.016394,1,68.623807,68.647390,0,100.000000,62.602106,0,0.069346,1,29.770000,30.980001,28.960000,31.790001,27.749999,33.000002,26.939999,0


In [4]:
dftouse=df.copy()

### Feature Engineering

In [5]:
IGNORE = ['date', 'result_1','close_1','perf_1','result_14','close_14','perf_14','results']

In [6]:
INDICATORS=[]
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10 and v not in IGNORE:
        print v, l
        INDICATORS.append(v)

bb_squeeze [False True]
bb_signalup [False True]
bb_signaldn [False True]
bb_signal [ 1.  0.]
ema_signal1 [0 1]
ema_signal2 [1 0]
kama_signal1 [0 1]
kama_signal2 [0 1]
sar_signal [0 1]
adx_trend [True False]
adx_direction [False True]
adx_signal [ 0.  1.]
aroon_signal [0 1]
cci_signal [ 1.  0.]
macd_signal [0 1]
ppo_signal [0 1]
mfi_signal [ 1.  0.]
roc_signal [ 0.  1.]
rsi_signal [ 1.  0.]
ult_signal [ 0.  1.]
wr_signal [ 1.  0.]
ad_signal [0 1]
sslow_signal [ 1.  0.]
srsi_signal [ 0.  1.]
trix_signal [1 0]
cv_signal [1 0]


In [7]:
STANDARDIZABLE = []
for v in df.columns:
    if v not in INDICATORS and v not in IGNORE:
        print v
        STANDARDIZABLE.append(v)

open
high
low
close
volume
bb_upper
bb_middle
bb_lower
bb_pct
bb_bandwidth
ema50
ema150
ema200
kama50
kama150
kama200
sar
adx
plus_di
minus_di
aroon_osc
cci
macd
macd_sigline
macd_hist
ppo
mfi
roc
rsi
ult_osc
willr
ad_osc
stoch_slowk
stoch_slowd
stoch_fastk
stoch_fastd
trix
sr_pivotpts
sr_res1
sr_sup1
sr_res2
sr_sup2
sr_res3
sr_sup3


In [8]:
# from sklearn.cross_validation import train_test_split
# itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
# mask=np.ones(dftouse.shape[0], dtype='int')
# mask[itrain]=1
# mask[itest]=0
# mask = (mask==1)
# mask.shape, mask.sum()

In [9]:
#dftouse['date'] = pd.to_datetime(dftouse['date'])
#mask = (dftouse.date < '2015-01-01').values
#mask.shape, mask.sum()

mask = dftouse['date'] < sdatev
maskv = (dftouse['date'] >= sdatev) & (dftouse['date'] <= edatev)
maskt = (dftouse['date'] >= sdatet) & (dftouse['date'] <= edatet)

### Check ROI of signals, alone:

In [10]:
#print "ROI baseline: 1.12%"
#print 'ROI "result" buy-only: 67.45%'
#print 'ROI "result" buy-sell: 172.49%'
    
def signalperf(signal, valtest):
    ypred = df[signal]
    if valtest == "v":
        df_pred = df[maskv].reset_index(drop=True)
        sdate = sdatev
        edate = edatev
    else:
        df_pred = df[maskt].reset_index(drop=True)
        sdate = sdatet
        edate = edatet
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])

    balance, profit, ROI2, balovertime, signals = evaluate_profit(df_pred, sdate, edate, 10000, 'pred_result', 'close', False, [1])
    #print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)

    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdate, edate, 10000, 'pred_result', 'close', False, [0,1])
    print signal + ': ROI "pred" buy-only: {0:.2f}%'.format(ROI2*100), 'buy-sell: {0:.2f}%'.format(ROI*100)
    return

In [11]:
print 'validation: ' + sdatev.strftime('%Y-%m-%d') + ' - ' + edatev.strftime('%Y-%m-%d')

for v in INDICATORS:
    signalperf(v, "v")

validation: 2013-01-01 - 2013-12-31
bb_squeeze: ROI "pred" buy-only: 0.00% buy-sell: -20.03%
bb_signalup: ROI "pred" buy-only: -1.96% buy-sell: -22.56%
bb_signaldn: ROI "pred" buy-only: 0.84% buy-sell: -15.98%
bb_signal: ROI "pred" buy-only: 20.03% buy-sell: 20.03%
ema_signal1: ROI "pred" buy-only: 10.17% buy-sell: 0.79%
ema_signal2: ROI "pred" buy-only: 20.03% buy-sell: 20.03%
kama_signal1: ROI "pred" buy-only: 9.26% buy-sell: -0.90%
kama_signal2: ROI "pred" buy-only: 22.84% buy-sell: 25.59%
sar_signal: ROI "pred" buy-only: 0.65% buy-sell: -16.88%
adx_trend: ROI "pred" buy-only: 2.57% buy-sell: -13.91%
adx_direction: ROI "pred" buy-only: 13.65% buy-sell: 6.85%
adx_signal: ROI "pred" buy-only: 13.07% buy-sell: 5.61%
aroon_signal: ROI "pred" buy-only: 20.36% buy-sell: 20.64%
cci_signal: ROI "pred" buy-only: 3.92% buy-sell: -11.27%
macd_signal: ROI "pred" buy-only: 3.67% buy-sell: -12.02%
ppo_signal: ROI "pred" buy-only: 17.10% buy-sell: 13.58%
mfi_signal: ROI "pred" buy-only: 5.70% buy-

In [12]:
print 'test: ' + sdatet.strftime('%Y-%m-%d') + '-' + edatet.strftime('%Y-%m-%d')

for v in INDICATORS:
    signalperf(v, "t")

test: 2014-01-01-2015-11-30
bb_squeeze: ROI "pred" buy-only: 0.00% buy-sell: -1.22%
bb_signalup: ROI "pred" buy-only: -7.38% buy-sell: -17.26%
bb_signaldn: ROI "pred" buy-only: 3.67% buy-sell: 4.86%
bb_signal: ROI "pred" buy-only: 1.22% buy-sell: 1.22%
ema_signal1: ROI "pred" buy-only: -1.77% buy-sell: -4.93%
ema_signal2: ROI "pred" buy-only: -0.10% buy-sell: -1.42%
kama_signal1: ROI "pred" buy-only: -1.88% buy-sell: -5.66%
kama_signal2: ROI "pred" buy-only: -3.68% buy-sell: -9.82%
sar_signal: ROI "pred" buy-only: 1.89% buy-sell: 1.49%
adx_trend: ROI "pred" buy-only: 5.85% buy-sell: 9.70%
adx_direction: ROI "pred" buy-only: 1.55% buy-sell: 1.07%
adx_signal: ROI "pred" buy-only: 5.37% buy-sell: 9.11%
aroon_signal: ROI "pred" buy-only: -3.63% buy-sell: -8.87%
cci_signal: ROI "pred" buy-only: 11.23% buy-sell: 21.13%
macd_signal: ROI "pred" buy-only: 1.68% buy-sell: 0.99%
ppo_signal: ROI "pred" buy-only: 1.72% buy-sell: 1.11%
mfi_signal: ROI "pred" buy-only: -4.31% buy-sell: -9.73%
roc_sig

#### 1.2 Standardize the data

Use the mask to compute the training and test parts of the dataframe. Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.

In [13]:
#your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dftouse[mask][STANDARDIZABLE])
dftouse[STANDARDIZABLE] = scaler.transform(dftouse[STANDARDIZABLE])
dftouse.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2006-01-03,-0.139470,-0.129461,-0.103412,-0.099306,0.018278,23.190001,True,0.009578,23.110001,True,0.006095,1,-0.010627,0.023020,0.056274,-1.236765,-0.301370,False,False,False,1,0.002681,-0.004804,-0.020142,0,1,-0.054202,-0.011372,-0.053049,0,0,-0.034574,0,0.349589,-1.814214,0.569255,True,False,0,-1.686859,0,-0.959991,1,-0.510036,-0.222454,-1.075150,0,-0.682402,0,-1.483684,1,-0.825522,0,-1.736132,1,-1.583818,0,-1.303566,1,-0.454272,0,-1.239042,-1.713160,1,1.154103,-0.551098,0,0.169181,1,-0.110760,-0.369818,-0.118723,-0.352518,0.126519,-0.586984,0.094418,1
1,2006-01-04,-0.092913,-0.076405,-0.052736,-0.052733,-0.526555,23.270000,True,0.003450,23.230000,True,0.001725,1,-0.022371,0.014386,0.050872,-0.753401,-0.320995,False,False,False,1,0.000525,-0.005475,-0.020532,0,1,-0.054355,-0.011566,-0.053140,0,0,-0.133971,1,0.282321,-0.920838,0.260030,True,False,0,-1.686859,0,-0.495968,1,-0.482572,-0.279325,-0.781605,0,-0.727840,0,-1.662264,1,-0.525831,0,-0.881402,1,-0.900575,0,-0.744774,1,-0.242742,1,-0.317250,-1.193324,1,1.154103,0.458445,0,0.147854,1,-0.060635,-0.272125,-0.027274,-0.303450,0.172279,-0.493407,0.173826,1
2,2006-01-05,-0.046357,-0.057305,-0.014730,-0.035797,-0.767793,23.480000,True,0.009024,23.469999,True,0.008595,1,-0.035403,0.006817,0.048870,-0.542979,-0.356958,False,False,False,1,-0.000862,-0.005886,-0.020721,0,1,-0.054204,-0.011682,-0.053139,0,0,-0.133971,1,0.159871,-0.683017,0.078830,True,False,0,-1.686859,0,-0.212678,1,-0.437984,-0.315440,-0.497440,0,-0.747148,0,-1.614582,1,-0.341751,0,-0.611734,1,-0.810298,0,-0.453530,1,-0.152175,1,0.622178,-0.335299,1,1.154103,1.467988,0,0.127482,1,-0.035926,-0.223967,0.017807,-0.279261,0.194837,-0.447278,0.212971,1
3,2006-01-06,-0.046357,-0.008494,-0.004173,0.008659,-0.264380,23.490000,True,0.000426,23.629999,True,0.006388,0,-0.043361,0.002980,0.049202,-0.084799,-0.384462,False,False,False,1,-0.000399,-0.005635,-0.020390,1,1,-0.053268,-0.011594,-0.052840,1,0,-0.128843,1,-0.016612,-0.042984,-0.241145,True,True,1,-1.558884,0,0.235623,1,-0.351838,-0.326207,-0.150395,0,-0.738002,0,-1.172559,1,-0.116182,0,0.012429,1,-0.399088,0,0.115417,1,-0.008713,1,1.167727,0.530791,0,1.154103,1.467988,0,0.109605,1,-0.001333,-0.156545,0.080919,-0.245397,0.226418,-0.382698,0.267773,1
4,2006-01-09,0.012898,-0.027594,0.012719,0.010776,-0.790551,23.440001,False,-0.002129,23.870001,True,0.016177,0,-0.049397,-0.000005,0.049309,-0.033172,-0.404840,False,False,False,1,0.000132,-0.005355,-0.020037,1,1,-0.052020,-0.011482,-0.052487,1,0,-0.118561,1,-0.180489,-0.236534,-0.397735,True,True,1,-1.558884,0,0.364546,1,-0.278190,-0.319324,0.091238,1,-0.726341,0,-1.120871,1,0.291819,0,0.039963,1,0.327271,0,0.142510,1,0.112874,1,1.211972,1.081090,0,1.154103,1.467988,0,0.094019,1,-0.001333,-0.156545,0.080919,-0.245397,0.226418,-0.382698,0.267773,1


We create a list `lcols` of the columns we will use in our classifier. This list should not contain the response `RESP`. How many features do we have?

In [14]:
#lcols=list(dftouse.columns)
#lcols.remove(u'results')
lcols=[]
#lcols.append('cv_signal')
for c in list(dftouse.columns):
    #if c not in INDICATORS and c not in IGNORE:
    #if c in INDICATORS:
    if c not in IGNORE:  #Original
        lcols.append(c)
print len(lcols)

70


### EDA for the data

We create a variable `ccols` which contains all variables not in our indicators list

In [15]:
ccols=[]
for c in lcols:
    if c not in INDICATORS and c not in IGNORE:
        ccols.append(c)
print len(ccols), len(INDICATORS)
ccols

44 26


['open',
 'high',
 'low',
 'close',
 'volume',
 'bb_upper',
 'bb_middle',
 'bb_lower',
 'bb_pct',
 'bb_bandwidth',
 'ema50',
 'ema150',
 'ema200',
 'kama50',
 'kama150',
 'kama200',
 'sar',
 'adx',
 'plus_di',
 'minus_di',
 'aroon_osc',
 'cci',
 'macd',
 'macd_sigline',
 'macd_hist',
 'ppo',
 'mfi',
 'roc',
 'rsi',
 'ult_osc',
 'willr',
 'ad_osc',
 'stoch_slowk',
 'stoch_slowd',
 'stoch_fastk',
 'stoch_fastd',
 'trix',
 'sr_pivotpts',
 'sr_res1',
 'sr_sup1',
 'sr_res2',
 'sr_sup2',
 'sr_res3',
 'sr_sup3']

#### 1.4 Train a SVM on this data.

In [16]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV

In [17]:
"""
Function
--------
cv_optimize

Inputs
------
clf : an instance of a scikit-learn classifier
parameters: a parameter grid dictionary thats passed to GridSearchCV (see above)
X: a samples-features matrix in the scikit-learn style
y: the response vectors of 1s and 0s (+ives and -ives)
n_folds: the number of cross-validation folds (default 5)
score_func: a score function we might want to pass (default python None)
   
Returns
-------
The best estimator from the GridSearchCV, after the GridSearchCV has been used to
fit the model.
     
Notes
-----
see do_classify and the code below for an example of how this is used
"""
#your code here
def cv_optimize(clf, parameters, X, y, n_folds, score_func):
    fitmodel = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    fitmodel.fit(X, y)
    return fitmodel.best_estimator_

In [18]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=mask, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf#.values
    y=(indf[targetname].values==target1val)*1
    y=indf[targetname]
    #if mask !=None:
    #    print "using mask"
    #    Xtrain, Xval, Xtest, ytrain, yval, ytest = X[mask], X[maskv], X[maskt], y[mask], y[maskv], y[maskt]
    Xtrain, Xval, Xtest, ytrain, yval, ytest = X[mask], X[maskv], X[maskt], y[mask], y[maskv], y[maskt]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xval, Xtest, ytrain, yval, ytest = reuse_split['Xtrain'], reuse_split['Xval'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['yval'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    val_accuracy = clf.score(Xval, yval)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on validation data: %0.2f" % (val_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xval, yval, Xtest, ytest

In [19]:
%%time
clfsvm, Xtrain, ytrain, Xval, yval, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]}, dftouse, lcols, u'results', 1, mask=mask)

############# based on standard predict ################
Accuracy on training data: 0.68
Accuracy on validation data: 0.67
Accuracy on test data:     0.71
[[199  63]
 [ 77 142]]
########################################################
CPU times: user 2.84 s, sys: 36.7 ms, total: 2.87 s
Wall time: 4.28 s


In [20]:
clfsvm

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [21]:
predsv={}
predsv_ROI_long={}
predsv_ROI_longshort={}

predst={}
predst_ROI_long={}
predst_ROI_longshort={}

In [22]:
df_pred = df[maskv].reset_index(drop=True)
df_pred['baseline'] = np.ones(df_pred.shape[0])

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'baseline', 'close', False, [1])
print "ROI baseline: {0:.2f}%".format(ROI*100)
predsv["baseline_long"] = df_pred['baseline']
predsv_ROI_long["baseline_long"] = ROI
predsv_ROI_longshort["baseline_long"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'results', 'close', False, [1])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predsv["baseline_ema"] = df_pred['results']
predsv_ROI_long["baseline_ema"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'results', 'close', False, [0,1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predsv_ROI_longshort["baseline_ema"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'result_1', 'close', False, [1])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predsv["baseline_max"] = df_pred['result_1']
predsv_ROI_long["baseline_max"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'result_1', 'close', False, [0,1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predsv_ROI_longshort["baseline_max"] = ROI

ROI baseline: 20.03%
ROI "result" buy-only: 48.84%
ROI "result" buy-sell: 81.63%
ROI "result" buy-only: 149.35%
ROI "result" buy-sell: 411.19%


In [23]:
df_pred = df[maskt].reset_index(drop=True)
df_pred['baseline'] = np.ones(df_pred.shape[0])

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'baseline', 'close', False, [1])
print "ROI baseline: {0:.2f}%".format(ROI*100)
predst["baseline_long"] = df_pred['baseline']
predst_ROI_long["baseline_long"] = ROI
predst_ROI_longshort["baseline_long"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'results', 'close', False, [1])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predst["baseline_ema"] = df_pred['results']
predst_ROI_long["baseline_ema"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'results', 'close', False, [0,1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predst_ROI_longshort["baseline_ema"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'result_1', 'close', False, [1])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predst["baseline_max"] = df_pred['result_1']
predst_ROI_long["baseline_max"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'result_1', 'close', False, [0,1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predst_ROI_longshort["baseline_max"] = ROI

ROI baseline: 1.22%
ROI "result" buy-only: 144.70%
ROI "result" buy-sell: 474.01%
ROI "result" buy-only: 534.80%
ROI "result" buy-sell: 3664.05%


In [24]:
def evaluate(clf, desc):

    #Validation
    ypred = clf.predict(Xval)
    df_pred = df[maskv].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    print "accuracy on validation set: {0:.3f}".format((df_pred.result_14 == df_pred.pred_result).sum()/float(len(df_pred)))
    
    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [1])
    print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
    predsv[desc] = ypred
    predsv_ROI_long[desc] = ROI

    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [0,1])
    print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
    predsv_ROI_longshort[desc] = ROI
    
    #Test
    ypred = clf.predict(Xtest)
    df_pred = df[maskt].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    print "accuracy on test set: {0:.3f}".format((df_pred.result_14 == df_pred.pred_result).sum()/float(len(df_pred)))
    
    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [1])
    print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
    predst[desc] = ypred
    predst_ROI_long[desc] = ROI

    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [0,1])
    print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
    predst_ROI_longshort[desc] = ROI

In [25]:
evaluate(clfsvm, "svm")

accuracy on validation set: 0.488
ROI "pred" buy-only: -0.74%
ROI "pred" buy-sell: -20.40%
accuracy on test set: 0.553
ROI "pred" buy-only: 14.86%
ROI "pred" buy-sell: 28.37%


The results obtained ought to be very similar to the efforts you put in earlier. If not its likely you wrote `cv_optimize` wrong. (Remember that we are using the same mask).

We'll reuse the training and test sets you computed above later in the homework. We do this by putting them into a dictionary `reuse_split`

In [26]:
reuse_split=dict(Xtrain=Xtrain, Xval=Xval, Xtest=Xtest, ytrain=ytrain, yval=yval, ytest=ytest)

## 2. Estimate costs and benefits from assumptions and data

### Our data is highly asymmetric

First notice that our data set is very highly asymmetric, with positive `RESP`onses only making up 16-17% of the samples.

In [27]:
print "whole data set", dftouse['results'].mean()#Highly asymmetric
print "training set", dftouse['results'][mask].mean(), "val set", dftouse['results'][maskv].mean(), "test set", dftouse['results'][maskt].mean()

whole data set 0.470328789094
training set 0.478137421919 val set 0.444444444444 test set 0.455301455301


This means that a classifier which predicts that EVERY customer is a negative has an accuracy rate of 83-84%. By this we mean that **a classifier that predicts that no customer will respond to our mailing** has an accuracy of 83-84%!

In [28]:
#your code here
from sklearn.linear_model import LogisticRegression
clflog = LogisticRegression(penalty="l1")
parameters = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clflog, Xtrain, ytrain, Xval, yval, Xtest, ytest = do_classify(clflog, parameters, dftouse, lcols, u'results',1, mask=mask)#, reuse_split=reuse_split)

############# based on standard predict ################
Accuracy on training data: 0.67
Accuracy on validation data: 0.67
Accuracy on test data:     0.72
[[190  72]
 [ 62 157]]
########################################################


In [29]:
clflog

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [30]:
evaluate(clflog, "logistic_regression")

accuracy on validation set: 0.496
ROI "pred" buy-only: 3.87%
ROI "pred" buy-sell: -11.72%
accuracy on test set: 0.565
ROI "pred" buy-only: 20.38%
ROI "pred" buy-sell: 41.67%


In [31]:
from sklearn.ensemble import RandomForestClassifier
clfraf = RandomForestClassifier(n_jobs=2)
parameters = {}#{"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clfraf, Xtrain, ytrain, Xval, yval, Xtest, ytest = do_classify(clfraf, parameters, dftouse, lcols, u'results',1, mask=mask, reuse_split=reuse_split)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.99
Accuracy on validation data: 0.71
Accuracy on test data:     0.66
[[178  84]
 [ 81 138]]
########################################################


In [32]:
evaluate(clfraf, "random_forest")

accuracy on validation set: 0.579
ROI "pred" buy-only: 15.03%
ROI "pred" buy-sell: 9.01%
accuracy on test set: 0.526
ROI "pred" buy-only: 4.76%
ROI "pred" buy-sell: 7.30%


## 4. Trying to improve the SVM: Feature Selection and Data Balancing

If you did the previous section right, you will find that the logistic regression model provides a better profit over some section of the profit curve than the baseline "send to everyone" classifier, while the SVM classifier is generally poor. At this might we might want to try all kinds of classifiers: from perceptrons to random forests. In the interest of time, and to study the SVM in some more detail, we'll restrict ourselves to trying to improve the SVM performance here. In real life you would try other classifiers as well.

 We wont be exhaustive in this improvement process either(which is something you should do on your project) in the interests of time, but we'll explore if feature-selection on the  SVM, and data balancing on the SVM (SVM's are known to perform better on balanced data) help.
 
( An aside: many classifiers such as SVM and decision trees struggle in their techniques on imbalanced data. You can read more at: see Weiss, Gary M., and Foster Provost. "The effect of class distribution on classifier learning: an empirical study." Rutgers Univ (2001). Also see http://pages.stern.nyu.edu/~fprovost/Papers/skew.PDF and http://www.cs.ox.ac.uk/people/vasile.palade/papers/Class-Imbalance-SVM.pdf for multiple ways to deal with the imbalance problem: balancing is not always the best option. `Sklearn` also provides a class weighting strategy: http://scikit-learn.org/stable/modules/svm.html#unbalanced-problems ). 

### Feature Selection

The Lasso, for example, implements internally, a form of feature selection by setting many coefficients to zero. Let us find coefficients that are non-zero.

#### Non zero lasso features

We write a function `nonzero_lasso` which takes the fit classifier `clfloglasso` as an argument, and spits out a dataframe of coefficients, sorted by the absolute magnitude of the coefficients. This way we can see which features dominated the logistic regression.

In [33]:
def nonzero_lasso(clf):
    featuremask=(clf.coef_ !=0.0)[0]
    return pd.DataFrame(dict(feature=lcols, coef=clf.coef_[0], abscoef=np.abs(clf.coef_[0])))[featuremask].sort('abscoef', ascending=False)

In [34]:
lasso_importances=nonzero_lasso(clflog)
lasso_importances.set_index("feature", inplace=True)
lasso_importances.head(10)

,abscoef,coef
feature,,
stoch_slowk,0.179758,-0.179758
stoch_fastd,0.145107,-0.145107
plus_di,0.122145,-0.122145
cci,0.092789,-0.092789
stoch_fastk,0.037305,-0.037305


#### 4.1 Feature importance using correlations

We can also get a notion of which features are important in the classification process by seeing how they correlate with the response. Implement some code to obtain the Pearson correlation coefficient between each of our features and the response. Do this on the training set only! Create a dataframe indexed by the features, which has columns `abscorr` the absolute value of the correlation and `corr` the value of the correlation. Sort the dataframe by `abscorr`, highest first, and show the top 25 features with the highest absolute correlation. Is there much overlap with the feature selection performed by the LASSO?

In [35]:
from scipy.stats.stats import pearsonr
correlations=[]
dftousetrain=dftouse[mask]
for col in lcols:
    r=pearsonr(dftousetrain[col], dftousetrain['results'])[0]
    correlations.append(dict(feature=col,corr=r, abscorr=np.abs(r)))

bpdf=pd.DataFrame(correlations).sort('abscorr', ascending=False)
bpdf.set_index(['feature'], inplace=True)
bpdf.head(25)

,abscorr,corr
feature,,
stoch_slowk,0.340986,-0.340986
cci,0.329036,-0.329036
stoch_fastd,0.324571,-0.324571
bb_pct,0.303100,-0.303100
willr,0.302728,-0.302728
rsi,0.291268,-0.291268
stoch_slowd,0.289560,-0.289560
plus_di,0.280720,-0.280720
srsi_signal,0.278185,0.278185


#### Why Feature Select?

One of the reasons feature selection is done, automatically or otherwise, is that there might be strong correlations between features. Also recall polynomial regression: a large number of features can lead to overfitting. Feature selection helps curb the problem of the curse of dimensionality, where centrality measures often used in statistics go wonky at higher dimensions. Between feature-engineering which we did some of, earlier, and feature selection, is where a lot of smarts and domain knowledge comes in. You will gain this with experience.

### Create a pipeline to feature-select, standardize and train!

We shall use sklearn pipelines to do correlation-with-response based feature selection for our SVM model. Maybe such feature-selection will improve the abysmal performance. 

This does not reduce the collinearity amongst the features, for which one either needs PCA, ICA, or some feature selection using the forward-backward algorithm. We do not have the time to approach it here. 

Its very important to do response based feature selection in the right way. If you remember, we separately standardized the training and test sets. This was to prevent **any** information about the overall mean and standard deviation leaking into the test set. 

But we played a bit loose with the rules there. We standardized on the entire training set. Instead we should have been standardizing separately in each cross-validation fold. There the original training set would be broken up into a sub-training and validation set, the standardization needed to be done on those separately. This can be implemented with `sklearn` pipelines.

Such kind of "data snooping" is relatively benign though, as it used no information about the response variable. But if you do any feature selection which uses the response variable, such as choosing the "k" most correlated variables from above, its not benign any more. This is because you have leaked the response from the validation into your sub-training set, and cannot thus be confident about your predictions: you might overfit. In such a situation, you must do the feature selection inside the cross-validation fold. See http://nbviewer.ipython.org/github/cs109/content/blob/master/lec_10_cross_val.ipynb from the 2013 course for a particularly dastardly case of this, where you see that the problem is particularly exacerbated when you have many more features than samples.

Lets do this here using sklearn pipelines.

In [36]:
from sklearn import feature_selection
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

Lets define a scorer which returns the absolute values of the pearson correlation between the feature and the response for each sample. The specific form of the scorer is dictated to us in the API docs for `SelectKBest`, see [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html): the first argument must be an array of scores, and the second an array of p-values.

In [37]:
def pearson_scorer(X,y):
    rs=np.zeros(X.shape[1])
    pvals=np.zeros(X.shape[1])
    i=0
    for v in X.T:
        rs[i], pvals[i]=pearsonr(v, y)
        i=i+1
    return np.abs(rs), pvals    

Lets apply the feature selection to a model which did not have any automatic feature selection and performed rather poorly before: the linear SVM. 

The `Pipeline` feature of sklearn chains various parts of a machine learning algorithm together. In this case we want to chain feature-selection and training in such a way that both happen freshly for each cross-validation fold (we wont bother to standardize in each cross-validation fold separately here for brevity, although you might want to do this).
We use the `SelectKBest` meta estimator to select the 25 most correlated/anti-correlated features. We create an instance of this meta-estimator, `selectorlinearsvm`. We then combine it with the linear SVC estimators into the pipeline `pipelinearsvm`: the `Pipeline` function simply takes a list of `scikit-learn` estimators and wraps them together into a new estimator object, which can then be passed to `GridSearchCV` via our `do_classify` function. Notice how this new estimator object can be used exactly the same way as a single classifier can be used in `scikit-learn`..this uniformity of interface is one of the nice features of `sklearn`!

In [38]:
selectorlinearsvm = SelectKBest(k=25, score_func=pearson_scorer)
pipelinearsvm = Pipeline([('select', selectorlinearsvm), ('svm', LinearSVC(loss="hinge"))])


#### Let us run the pipelined classifier 

We'll run the classifier and compare the results using the ROC curve to the previous SVM result.

In [39]:
pipelinearsvm, _,_,_,_,_,_  = do_classify(pipelinearsvm, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.67
Accuracy on validation data: 0.69
Accuracy on test data:     0.72
[[197  65]
 [ 68 151]]
########################################################


What features did the pipelined classifier use? We can access them so:

In [40]:
np.array(lcols)[pipelinearsvm.get_params()['select'].get_support()]

array(['bb_pct', 'ema_signal1', 'sar_signal', 'plus_di', 'minus_di',
       'adx_direction', 'aroon_osc', 'cci', 'macd', 'macd_hist',
       'macd_signal', 'mfi', 'roc', 'rsi', 'ult_osc', 'willr', 'wr_signal',
       'ad_osc', 'ad_signal', 'stoch_slowk', 'stoch_slowd', 'sslow_signal',
       'stoch_fastk', 'stoch_fastd', 'srsi_signal'], 
      dtype='|S13')

We plot the ROC curves, using the label `svm-feature-selected` for the pipelined classifier `pipelinearsvm`. We plot it alongside the older logistic-with lasso and all-features SVM for comparison

In [41]:
evaluate(pipelinearsvm, "pipelinearsvm")

accuracy on validation set: 0.480
ROI "pred" buy-only: 1.68%
ROI "pred" buy-sell: -16.04%
accuracy on test set: 0.576
ROI "pred" buy-only: 24.05%
ROI "pred" buy-sell: 50.44%


#### 4.3 Implement a RBF based pipelined (feature-selected) classifier on the balanced set.

In [42]:
from sklearn.svm import SVC
reuse_split_new = reuse_split

In [43]:
%%time
selectorsvm2 = SelectKBest(k=25, score_func=pearson_scorer)
pipesvm2 = Pipeline([('select2', selectorsvm2), ('svm2', SVC())])
pipesvm2, _,_,_,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'results',1, reuse_split=reuse_split)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.66
Accuracy on validation data: 0.69
Accuracy on test data:     0.73
[[194  68]
 [ 64 155]]
########################################################
CPU times: user 7.98 s, sys: 71.2 ms, total: 8.05 s
Wall time: 8.81 s


In [44]:
pipesvm2.get_params()

{'select2': SelectKBest(k=25, score_func=<function pearson_scorer at 0x1095ec1b8>),
 'select2__k': 25,
 'select2__score_func': <function __main__.pearson_scorer>,
 'svm2': SVC(C=100000000.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
   gamma=1e-09, kernel='rbf', max_iter=-1, probability=False,
   random_state=None, shrinking=True, tol=0.001, verbose=False),
 'svm2__C': 100000000.0,
 'svm2__cache_size': 200,
 'svm2__class_weight': None,
 'svm2__coef0': 0.0,
 'svm2__degree': 3,
 'svm2__gamma': 1e-09,
 'svm2__kernel': 'rbf',
 'svm2__max_iter': -1,
 'svm2__probability': False,
 'svm2__random_state': None,
 'svm2__shrinking': True,
 'svm2__tol': 0.001,
 'svm2__verbose': False}

In [45]:
evaluate(pipesvm2, "pipesvm2")

accuracy on validation set: 0.516
ROI "pred" buy-only: 1.18%
ROI "pred" buy-sell: -16.68%
accuracy on test set: 0.586
ROI "pred" buy-only: 28.53%
ROI "pred" buy-sell: 61.97%


## Yvan ML

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score

# selector of  features
featSelector = SelectKBest(k=18, score_func=pearson_scorer)

print "#############====================== Log Regression =====================#############"
pipeLR = Pipeline([('select', featSelector), ('LR', LogisticRegression(penalty="l1"))])
pipeLR, _,_,_,_,_,_  = do_classify(pipeLR, {"LR__C": [0.005, 0.01, 0.02, 0.05, 10.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeLR, "pipe_Log_Regr")
print "#############====================== Linear SVM ========================#############"
clfsvm_b = Pipeline([('select', featSelector), ('svm', LinearSVC(loss="hinge"))])
clfsvm_b, _,_,_,_,_,_  = do_classify(clfsvm_b, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(clfsvm_b, "linear_svm")
print "#############====================== RBF SVC ===========================#############"
pipesvm2 = Pipeline([('select', featSelector), ('svm2', SVC())])
pipesvm2, _,_,_,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipesvm2, "rbf_svc")
print "#############====================== Random Forest =====================#############"
pipeRF = Pipeline([('select', featSelector), ('RF', RandomForestClassifier())])
pipeRF, _,_,_,_,_,_  = do_classify(pipeRF, {"RF__max_depth": [3,5,7,10,15,25,50], "RF__n_estimators": [5,10,20,40],"RF__max_features": [1,2,3]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeRF, "rand_forest")
print "#############====================== Extra Trees= =====================#############"
pipeET = Pipeline([('select', featSelector), ('ET', RandomForestClassifier())])
pipeET, _,_,_,_,_,_  = do_classify(pipeET, {"ET__max_depth": [3,5,7,10,15,25,50], "ET__n_estimators": [5,10,20,40],"ET__max_features": [1,2,3]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeET, "extra_trees")
print "#############====================== AdaBoost ==========================#############"
pipeAda = Pipeline([('select', featSelector), ('Ada', AdaBoostClassifier())])
pipeAda, _,_,_,_,_,_  = do_classify(pipeAda, {"Ada__n_estimators": [5,10,20,40],"Ada__learning_rate": [0.1,0.5,1.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeAda, "ada_boost")
print "#############====================== Gaussian NB ==========================#############"
pipeNB = Pipeline([('select', featSelector), ('NB', GaussianNB())])
pipeNB, _,_,_,_,_,_  = do_classify(pipeNB, {}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeNB, "gaussian_nb")
print "#############====================== Gradient Boosting ====================#############"
pipeGB = Pipeline([('select', featSelector), ('GB', GradientBoostingClassifier())])
pipeGB, _,_,_,_,_,_  = do_classify(pipeGB, {"GB__n_estimators": [5,10,20,40],"GB__learning_rate": [0.1,0.5,1.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeGB, "gradient_boosting")

#############====================== Log Regression =====================#############
using reuse split
############# based on standard predict ################
Accuracy on training data: 0.67
Accuracy on validation data: 0.68
Accuracy on test data:     0.72
[[190  72]
 [ 65 154]]
########################################################
accuracy on validation set: 0.496
ROI "pred" buy-only: 3.23%
ROI "pred" buy-sell: -12.92%
accuracy on test set: 0.551
ROI "pred" buy-only: 11.29%
ROI "pred" buy-sell: 21.11%
#############====================== Linear SVM ========================#############
using reuse split
############# based on standard predict ################
Accuracy on training data: 0.67
Accuracy on validation data: 0.69
Accuracy on test data:     0.72
[[195  67]
 [ 68 151]]
########################################################
accuracy on validation set: 0.484
ROI "pred" buy-only: 1.68%
ROI "pred" buy-sell: -16.04%
accuracy on test set: 0.572
ROI "pred" buy-only: 23.17%
ROI

## Ensemble

In [47]:
predsv.keys()

['svm',
 'pipelinearsvm',
 'rbf_svc',
 'linear_svm',
 'ada_boost',
 'rand_forest',
 'pipesvm2',
 'extra_trees',
 'logistic_regression',
 'baseline_ema',
 'pipe_Log_Regr',
 'random_forest',
 'gaussian_nb',
 'gradient_boosting',
 'baseline_max',
 'baseline_long']

In [48]:
for k in predsv_ROI_long:
    print k + ' ROI buy-only: {0:.2f}%'.format(predsv_ROI_long[k]*100)

svm ROI buy-only: -0.74%
pipelinearsvm ROI buy-only: 1.68%
rbf_svc ROI buy-only: 0.29%
linear_svm ROI buy-only: 1.68%
ada_boost ROI buy-only: 5.42%
rand_forest ROI buy-only: 4.32%
pipesvm2 ROI buy-only: 1.18%
extra_trees ROI buy-only: 1.82%
logistic_regression ROI buy-only: 3.87%
baseline_ema ROI buy-only: 48.84%
pipe_Log_Regr ROI buy-only: 3.23%
random_forest ROI buy-only: 15.03%
gaussian_nb ROI buy-only: -1.69%
gradient_boosting ROI buy-only: 7.45%
baseline_max ROI buy-only: 149.35%
baseline_long ROI buy-only: 20.03%


In [49]:
for k in predsv_ROI_longshort:
    print k + ' ROI buy-sell: {0:.2f}%'.format(predsv_ROI_longshort[k]*100)

svm ROI buy-sell: -20.40%
pipelinearsvm ROI buy-sell: -16.04%
rbf_svc ROI buy-sell: -18.04%
linear_svm ROI buy-sell: -16.04%
ada_boost ROI buy-sell: -9.20%
rand_forest ROI buy-sell: -11.24%
pipesvm2 ROI buy-sell: -16.68%
extra_trees ROI buy-sell: -15.37%
logistic_regression ROI buy-sell: -11.72%
baseline_ema ROI buy-sell: 81.63%
pipe_Log_Regr ROI buy-sell: -12.92%
random_forest ROI buy-sell: 9.01%
gaussian_nb ROI buy-sell: -21.17%
gradient_boosting ROI buy-sell: -5.67%
baseline_max ROI buy-sell: 411.19%
baseline_long ROI buy-sell: 20.03%


In [50]:
predst.keys()

['svm',
 'pipelinearsvm',
 'rbf_svc',
 'linear_svm',
 'ada_boost',
 'rand_forest',
 'pipesvm2',
 'extra_trees',
 'logistic_regression',
 'baseline_ema',
 'pipe_Log_Regr',
 'random_forest',
 'gaussian_nb',
 'gradient_boosting',
 'baseline_max',
 'baseline_long']

In [51]:
for k in predst_ROI_long:
    print k + ' ROI buy-only: {0:.2f}%'.format(predst_ROI_long[k]*100)

svm ROI buy-only: 14.86%
pipelinearsvm ROI buy-only: 24.05%
rbf_svc ROI buy-only: 29.41%
linear_svm ROI buy-only: 23.17%
ada_boost ROI buy-only: 12.96%
rand_forest ROI buy-only: 4.72%
pipesvm2 ROI buy-only: 28.53%
extra_trees ROI buy-only: 17.52%
logistic_regression ROI buy-only: 20.38%
baseline_ema ROI buy-only: 144.70%
pipe_Log_Regr ROI buy-only: 11.29%
random_forest ROI buy-only: 4.76%
gaussian_nb ROI buy-only: 27.42%
gradient_boosting ROI buy-only: 4.20%
baseline_max ROI buy-only: 534.80%
baseline_long ROI buy-only: 1.22%


In [52]:
for k in predst_ROI_longshort:
    print k + ' ROI buy-sell: {0:.2f}%'.format(predst_ROI_longshort[k]*100)

svm ROI buy-sell: 28.37%
pipelinearsvm ROI buy-sell: 50.44%
rbf_svc ROI buy-sell: 64.00%
linear_svm ROI buy-sell: 48.27%
ada_boost ROI buy-sell: 24.81%
rand_forest ROI buy-sell: 7.25%
pipesvm2 ROI buy-sell: 61.97%
extra_trees ROI buy-sell: 35.04%
logistic_regression ROI buy-sell: 41.67%
baseline_ema ROI buy-sell: 474.01%
pipe_Log_Regr ROI buy-sell: 21.11%
random_forest ROI buy-sell: 7.30%
gaussian_nb ROI buy-sell: 59.23%
gradient_boosting ROI buy-sell: 6.40%
baseline_max ROI buy-sell: 3664.05%
baseline_long ROI buy-sell: 1.22%


In [53]:
dfensemble=pd.DataFrame.from_dict({'svm':predsv['svm'],
                                     'pipelinearsvm':predsv['pipelinearsvm'],
                                     'rbf_svc':predsv['rbf_svc'],
                                     'linear_svm':predsv['linear_svm'],
                                     'ada_boost':predsv['ada_boost'],
                                     'rand_forest':predsv['rand_forest'],
                                     'pipesvm2':predsv['pipesvm2'],
                                     'extra_trees':predsv['extra_trees'],
                                     'logistic_regression':predsv['logistic_regression'],
                                     'pipe_Log_Regr':predsv['pipe_Log_Regr'],
                                     'random_forest':predsv['random_forest'],
                                     'gaussian_nb':predsv['gaussian_nb'],
                                     'gradient_boosting':predsv['gradient_boosting'],
                                   'y':predsv['baseline_ema']})

In [54]:
from sklearn.linear_model import LinearRegression
X = dfensemble.drop('y', axis = 1)
lm = LinearRegression()
valreg = lm.fit(X, dfensemble.y)


In [55]:
dfensembletest=pd.DataFrame.from_dict({'svm':predst['svm'],
                                     'pipelinearsvm':predst['pipelinearsvm'],
                                     'rbf_svc':predst['rbf_svc'],
                                     'linear_svm':predst['linear_svm'],
                                     'ada_boost':predst['ada_boost'],
                                     'rand_forest':predst['rand_forest'],
                                     'pipesvm2':predst['pipesvm2'],
                                     'extra_trees':predst['extra_trees'],
                                     'logistic_regression':predst['logistic_regression'],
                                     'pipe_Log_Regr':predst['pipe_Log_Regr'],
                                     'random_forest':predst['random_forest'],
                                     'gaussian_nb':predst['gaussian_nb'],
                                     'gradient_boosting':predst['gradient_boosting'],
                                   'y':predst['baseline_ema']})

In [56]:
Xt = dfensembletest.drop('y', axis = 1)
epreds = valreg.predict(Xt)

### Ensemble Results

In [57]:
#Test
df_pred = df[maskt].reset_index(drop=True)
df_pred['pred_result'] = 1*(epreds>0.5)
df_pred.head()

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'pred_result', 'close', False, [1])
print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
#predst[desc] = ypred
#predst_ROI_long[desc] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'pred_result', 'close', False, [0,1])
print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
#predst_ROI_longshort[desc] = ROI

ROI "pred" buy-only: 0.58%
ROI "pred" buy-sell: -1.02%
